In [1]:
import os
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import IPython.display as display
from itertools import product
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

class NeuralNetworkLayer(nn.Module):
    def __init__(self):
        super(NeuralNetworkLayer, self).__init__()

        #Layer 1 Input: 10 Output: 100
        self.layer1 = nn.Linear(10, 50)
        #nn.init.kaiming_normal_(self.layer1.weight, mode='fan_in', nonlinearity='relu')
        #nn.init.uniform_(self.layer1.weight, -0.7, 0.7)
        #nn.init.constant_(self.layer1.bias, 0.01)
        #Layer 2 Input: 50 Output: 100
        self.layer2 = nn.Linear(50, 100)
        #nn.init.kaiming_normal_(self.layer2.weight, mode='fan_in', nonlinearity='relu')
        #nn.init.uniform_(self.layer2.weight, -0.7, 0.7)
        #nn.init.constant_(self.layer2.bias, 0.01)
        #Layer 3 Input: 100 Output: 25
        self.layer3 = nn.Linear(100, 25)
        #nn.init.kaiming_normal_(self.layer2.weight, mode='fan_in', nonlinearity='relu')
        #nn.init.uniform_(self.layer3.weight, -0.7, 0.7)
        #nn.init.constant_(self.layer3.bias, 0.01)
        #Layer 4 Input: 25 Output: 3
        self.layer4 = nn.Linear(25, 3)
        #nn.init.kaiming_normal_(self.layer2.weight, mode='fan_in', nonlinearity='relu')
        #nn.init.uniform_(self.layer4.weight, -0.7, 0.7)
        #nn.init.constant_(self.layer4.bias, 0.01)

    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        x = F.relu(x)
        x = self.layer3(x)
        x = F.relu(x)
        x = self.layer4(x)
        return x

In [2]:
def importDatasetCupInput(file_name:str, blind:bool) -> pd.DataFrame:
    dataset = []
    try:
        dataset = pd.read_csv(file_name, header=None, dtype=float)
    except Exception as e:
        print("Error | Can not read dataset cup for take input")
        exit(1)
    if not blind:
        dataset = dataset.iloc[:, :-3] # Remove 3 columns
    columns_name = ['ID'] + [f'X{i}' for i in range(1,11)]
    dataset.columns = columns_name
    dataset.set_index('ID', inplace=True)
    return dataset

def importDatasetCupOutput(file_name:str, blind:bool) -> pd.DataFrame:
    try:
        dataset = pd.read_csv(file_name, header=None, dtype=float)
    except Exception as e:
        print("Error | Can not read dataset cup for take output")
        exit(1)
    columns_list = ['ID', 'Y1', 'Y2', 'Y3']
    if not blind: # Dataset with all inputs 
        indexes = [0, 10, 11, 12] # take the first and last 3 columns indexes
        dataset = dataset.iloc[:, indexes]
    dataset.columns = columns_list
    dataset.set_index('ID', inplace=True)
    return dataset

In [3]:
# GridSearch Params
results = []
hidden_sizes = [8,7,6,5]
learning_rates = [0.01, 0.001, 0.05, 0.005]
rates_split = [0.2, 0.25, 0.3]

#
learning_rate = 0.01
num_epochs = 300
threshold = 0.00001

## TRAIN
x_train = importDatasetCupInput("CUP/ML-CUP23-TRAIN.csv", blind=False)
y_train = importDatasetCupOutput("CUP/ML-CUP23-TRAIN.csv", blind=False)

x_train = torch.tensor(x_train.to_numpy())
y_train = torch.tensor(y_train.to_numpy())

x_train = x_train.to("cuda:0")
y_train = y_train.to("cuda:0")

x_train = x_train.double()
y_train = y_train.double()

## TEST
x_test = importDatasetCupInput("CUP/ML-CUP23-TEST-INPUT.csv", blind=True)
y_test = importDatasetCupOutput("CUP/ML-CUP23-TEST-TARGET.csv", blind=True)

x_test = torch.tensor(x_test.to_numpy())
y_test = torch.tensor(y_test.to_numpy())

x_test = x_test.to("cuda:0")
y_test = y_test.to("cuda:0")

x_test = x_test.double()
y_test = y_test.double()

dataset_train = TensorDataset(x_train, y_train)
data_loader_train = DataLoader(dataset_train, batch_size=500, shuffle=True)

dataset_test = TensorDataset(x_test, y_test)
data_loader_test = DataLoader(dataset_test, batch_size=450, shuffle=True)

In [5]:
net = NeuralNetworkLayer()
net = net.to("cuda:0")
net = net.double()

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)

#Model save 
torch.save(net, 'modelsCup/model1.pth')
with open('modelsCup/model1_parameters.txt', 'w') as file:
    file.write('Pesi layer1\n')
    file.write(str(net.layer1.weight.data) + '\n')
    file.write('Bias layer1\n')
    file.write(str(net.layer1.bias.data) + '\n')
    file.write('Pesi layer2\n')
    file.write(str(net.layer2.weight.data) + '\n')
    file.write('Bias layer2\n')
    file.write(str(net.layer2.bias.data) + '\n')
    file.write('Pesi layer3\n')
    file.write(str(net.layer3.weight.data) + '\n')
    file.write('Bias layer3\n')
    file.write(str(net.layer3.bias.data) + '\n')
    file.write('Pesi layer4\n')
    file.write(str(net.layer4.weight.data) + '\n')
    file.write('Bias layer4\n')
    file.write(str(net.layer4.bias.data) + '\n')

#Values used for graphs
loss_values_train = []
accuracy_values_train = []
loss_values_test = []
accuracy_values_test = []

net.train()
for epoch in range(num_epochs):
    total_loss = 0
    total = 0
    correct = 0

    for batch_input, batch_output in data_loader_train:
        #Forward pass
        outputs = net(batch_input)
        #Training loss
        loss = criterion(outputs, batch_output)
        #Calculate total loss
        total_loss += loss.item()
        #Backward and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        difference = torch.abs(outputs - batch_output)
        total += batch_input.size(0)
        correct += torch.sum(difference < threshold)
    accuracy = correct / total
    accuracy_values_train.append(accuracy.item())
    avg_loss = total_loss / len(data_loader_train)
    #Add to list
    loss_values_train.append(avg_loss)


    total = 0
    correct = 0
    #CALCULATE ACCURACY VAL
    net.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_input, batch_output in data_loader_test:
            outputs = net(batch_input)
            loss = criterion(outputs, batch_output)
            total_loss += loss.item()
            
            difference = torch.abs(outputs - batch_output)
            total += batch_input.size(0)
            correct += torch.sum(difference < threshold)
        accuracy = correct / total
        accuracy_values_test.append(accuracy.item())
        avg_loss = total_loss / len(data_loader_test)
        loss_values_test.append(avg_loss)
    net.train()

#Create dir
path_name = f'modelsCup/Cup-{threshold}-{learning_rate}'
os.makedirs(path_name, exist_ok=True)
#Save plot loss
display.clear_output(wait=True)
plt.plot(loss_values_train, label='Training Loss')
plt.plot(loss_values_test, label = 'Test loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss per Epoch')
plt.legend()
plt.savefig(f'{path_name}/Loss.png')
plt.clf()

#Save plot accuracy
display.clear_output(wait=True)
plt.plot(accuracy_values_train, label='Accuracy Train')
plt.plot(accuracy_values_test, label='Accuracy Test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy for Epoch')
plt.legend()
plt.savefig(f'{path_name}/Accuracy-test.png')
plt.clf()
    

<Figure size 640x480 with 0 Axes>

In [ ]:
for learning_rate, rate in product(learning_rates, rates_split):

    dataset_train = TensorDataset(train_data, train_target)
    data_loader_train = DataLoader(dataset_train, batch_size=100, shuffle=True)

    dataset_val = TensorDataset(val_data, val_target)
    data_loader_val = DataLoader(dataset_val, batch_size=100, shuffle=True)

    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    loss_values = []
    accuracy_values = []
    epochs = 390
    for epoch in range(epochs):
        total_loss = 0
        for batch_input, batch_output in data_loader_train:
            #Forward pass
            outputs = net(batch_input)
            #Training loss
            loss = criterion(outputs, batch_output)
            #Calculate total loss
            total_loss += loss.item()
            #Backward and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        avg_loss = total_loss / len(data_loader_train)
        #Add to list
        loss_values.append(avg_loss)

        total = 0
        correct = 0
        net.eval()
        with torch.no_grad():
            for batch_input, batch_output in data_loader_val:
                predicted = net(batch_input)
                total += batch_input.size(0)
                correct += (predicted == batch_output).sum().item()
            accuracy = correct / total
            accuracy_values.append(accuracy)

        net.train()

    #Create dir
    path_name = f'CUP/GridPlot/test-{hidden_size}-{learning_rate}-{rate}'
    os.mkdir(path_name)
    #Save plot loss
    display.clear_output(wait=True)
    plt.plot(loss_values, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss per Epoch')
    plt.legend()
    plt.savefig(f'{path_name}/Loss.png')
    plt.clf()
    #Save plot accuracy
    display.clear_output(wait=True)
    plt.plot(accuracy_values, label='Accuracy Test')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy for Epoch')
    plt.legend()
    plt.savefig(f'{path_name}/Accuracy-test.png')
    plt.clf()
    
    total = 0
    correct = 0
    net.eval()
    with torch.no_grad():
        for batch_input, batch_output in data_loader_test:
            predicted = net(batch_input)
            total += batch_input.size(0)
            correct += (predicted == batch_output).sum().item()

    accuracy = correct / total
    result = f'Hidden_size:{hidden_size}, Learning-rate:{learning_rate}, Rate: {rate} ,Accuracy: {accuracy:.4f}'
    print(result)
    results.append(result)

with open("resultsCUP.txt", 'w') as file:
    for result in results:
        file.write(result + "\n")